Data needed is of:

Free cash flow, Net debt, Growth rate, Current market price(CMP), number of shares, terminal growth rate




All of the required data is available from the Yahoo Finance Library and can be fetched easily 

**Understanding with the example of Blackstone (BX)** 

In [57]:
#getting all the required data

import yfinance as yf
from yahoo_fin import stock_info as si

# Create a Ticker object for the company
ticker_symbol = "BX"
company = yf.Ticker(ticker_symbol)

# Get the casflow statement for free cash flow
cash_flow = company.cashflow.loc['Free Cash Flow']
free_Cash_flow=cash_flow.iloc[0]
free_Cash_flow_cr=free_Cash_flow/10000000
print(f"Free cash flow of {ticker_symbol} is: {free_Cash_flow_cr} cr")

#net debt of the company
balancesheet = company.balancesheet

try:
    net_debt = balancesheet.loc['Net Debt'].iloc[0]
    net_debt_cr=net_debt/10000000
    print(F"Net Debt OF {ticker_symbol} is: {net_debt_cr} cr")

except Exception as e:
    print("Error:", str(e))

#growth rate of the company from the analysis section of the Yahoo finance
analysis = si.get_analysts_info(ticker_symbol)

# Retrieve the growth rate as a string
Growth_Estimates = analysis['Growth Estimates']
growth_dict = Growth_Estimates.to_dict()
growth_rate_str = growth_dict[ticker_symbol][4]

# Remove the percentage symbol and convert to a float
growth_rate_float = float(growth_rate_str.strip('%'))

# Convert the percentage to its decimal equivalent
growth_rate = growth_rate_float / 100

#growth rate for the first five years is same 
growth_rate_first_5_years=growth_rate
print(f"Growth rate for first 5 years of {ticker_symbol} is: {growth_rate_first_5_years*100}%")

#growth rate for the next five years is half of the prior growth rate
growth_rate_next_5_years =growth_rate_first_5_years/2
print(f"Growth rate for next 5 years of {ticker_symbol} is: {growth_rate_next_5_years * 100}%")

# Getting the current market price (CMP)
current_price = company.history(period="1d")["Close"].iloc[-1]
print(f"Current market price (CMP) for {ticker_symbol}: {current_price:.2f}")

# the number of shares outstanding
no_of_shares = company.info.get("sharesOutstanding")
no_of_shares_cr=no_of_shares/10000000
if no_of_shares is not None:
    print(f"Number of Shares of {ticker_symbol}: {no_of_shares_cr} cr")
else:
    print(f"Number of Shares data not available for {ticker_symbol}")

# Terminal growth rate (5%) (the default rate at which the economy of the country is growing)
terminal_rate = 0.05   

# Number of years on which we want to perform the dcf
years = 10             



Free cash flow of BX is: 610.0756 cr
Net Debt OF BX is: 818.7525 cr
Growth rate for first 5 years of BX is: 11.26%
Growth rate for next 5 years of BX is: 5.63%
Current market price (CMP) for BX: 112.92
Number of Shares of BX: 70.9750016 cr


**Discount rate data**

#specified earlier
The discount rate for BLACKSTONE (BX) is 13.30%

**Now after collecting the necessary let's data dive deep in the main part of the DCF valuation i.e forcasting the future free cash flow** 

First forcast the future cash flow with the "Time value of money" 

The time value of money (TVM) is the concept that a sum of money is worth more now than the same sum will be at a future date due to its earnings potential in the interim. (Investopedia)

Then get the present value of the money from forcasted cash flow and calculating the termianl value that is value beyond the forcasted period with the help of terminal growth rate and also getting the present value of the termianl vlaue using the same discount rate.

In [58]:

# Initialize the total future value, total present value, terminal value, and present value of the terminal value

total_future_value = 0
total_present_value = 0
terminal_value = 0
present_value_terminal = 0
discount_rate=0.12 #13.3%

# Calculate the future value and present value for each year, and calculate the total the totals
for year in range(1, years + 1):

    #first calculating for the first five years
    if year <= 5:
        future_value = free_Cash_flow_cr * (1 + growth_rate_first_5_years) ** year
        cash_flow_year_5 =free_Cash_flow_cr * (1 + growth_rate_first_5_years) **5

    #for next five years at different growth rate
    
    else:
        future_value = cash_flow_year_5 * (1 + growth_rate_next_5_years) ** (year-5)
    total_future_value += future_value

    #Calculating the present value of the future cash with the discount rate
    
    present_value_year = future_value / ((1 + discount_rate) ** year)
    total_present_value += present_value_year
    print(f"Year {year}: Future Value = {future_value:.2f} cr, Present Value = {present_value_year:.2f} cr")
    
# Calculate the terminal value
cash_flow_year_10 = free_Cash_flow_cr * (1 + growth_rate_next_5_years) ** years  # Using the second interest rate for year 10
terminal_value = (cash_flow_year_10 * (1 + terminal_rate)) / (discount_rate - terminal_rate)

# Calculate the present value of the terminal value
present_value_terminal = terminal_value / ((1 + discount_rate) ** years)


# Display the total future value, total present value, terminal value, and present value of the terminal value
print(f"\nTotal Future Value after {years} years = {total_future_value:.2f} cr")
print(f"\nTotal Present Value at a discount rate of {discount_rate*100:.2f}% = {total_present_value:.2f} cr")
print(f"\nTerminal Value at a terminal rate of {terminal_rate * 100}% = {terminal_value:.2f} cr")
print(f"\nPresent Value of Terminal Value = {present_value_terminal:.2f} cr")

Year 1: Future Value = 678.77 cr, Present Value = 606.04 cr
Year 2: Future Value = 755.20 cr, Present Value = 602.04 cr
Year 3: Future Value = 840.24 cr, Present Value = 598.06 cr
Year 4: Future Value = 934.85 cr, Present Value = 594.11 cr
Year 5: Future Value = 1040.11 cr, Present Value = 590.19 cr
Year 6: Future Value = 1098.67 cr, Present Value = 556.62 cr
Year 7: Future Value = 1160.52 cr, Present Value = 524.96 cr
Year 8: Future Value = 1225.86 cr, Present Value = 495.10 cr
Year 9: Future Value = 1294.88 cr, Present Value = 466.95 cr
Year 10: Future Value = 1367.78 cr, Present Value = 440.39 cr

Total Future Value after 10 years = 10396.86 cr

Total Present Value at a discount rate of 12.00% = 5474.46 cr

Terminal Value at a terminal rate of 5.0% = 15825.15 cr

Present Value of Terminal Value = 5095.27 cr


**Now get the totsl present value including the present value of the terminal value**

In [59]:
# Calculate and display the total present value including the terminal value
total_present_value_with_terminal = total_present_value + present_value_terminal
print(f"Total Present Value of free cash flows including Terminal Value = {total_present_value_with_terminal:.2f} cr")


Total Present Value of free cash flows including Terminal Value = 10569.74 cr


**Get the net present value of free cash floe i.e total-net debt**

In [60]:
#subtract the value of the net debt from the total present value of free cash flows

net_present_value_of_free_cash_flows= total_present_value_with_terminal-net_debt_cr
print(f"Net present value of free cash flows of {ticker_symbol} is: {net_present_value_of_free_cash_flows:.2f} cr" )

Net present value of free cash flows of BX is: 9750.98 cr


**Now, the ending part calculating the fair value of stock i.e net present value/no of shares**

In [61]:

fair_value=net_present_value_of_free_cash_flows/no_of_shares_cr
print(f"Fair value of the {ticker_symbol} is: ${fair_value:.2f}")


Fair value of the BX is: $137.39


**Conclusion**

In [62]:
if current_price>fair_value:
    print(f"{ticker_symbol} is overvalued according to DCF analysis")

else:
    print(f"{ticker_symbol} is undervalued according to DCF analysis")

BX is undervalued according to DCF analysis
